In [16]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline
np.set_printoptions(precision =4, suppress=True)

In [17]:
import PIL.Image
from io import StringIO
from io import BytesIO
import IPython.display

def showArray(a, fmt='png'):
    """Display an array image RAW without any resizing."""
    if a.ndim==2:
        a=np.uint8(a)
    elif a.ndim==3 and a.shape[2]==3:
        pass
    else:
        raise Exception('only 2d and 3d arrays with 3 colors supported')
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [18]:
USE_BLUE_CHANNEL = False

In [19]:
def grayscale(img):
    """Applies the Grayscale transform"""
    global USE_BLUE_CHANNEL
    if USE_BLUE_CHANNEL:
        r,g,b = cv2.split(img)
        return b
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """Applies a polygon image mask defined by vertices image mask."""
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, thickness=2, color=[255, 0, 0]):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """Detect Hough lines"""
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines


def weighted_img(initial_img, x, img, y, z):
    """initial_img * z + img * y + z"""
    return cv2.addWeighted(initial_img, x, img, y, z)

In [20]:

def mb2Hessian(m,b):
    '''returns rho and theta (from y=mx+b)'''
    D = np.sqrt(m**2 + 1)
    p =  math.fabs(b)/D
    theta = math.atan2(-1,m)
    return np.array([p,theta])

def linesPoints2MB(lines):
    '''fit y=mx+b line over two points, return (m,b)'''
    linesPoints = np.reshape(lines,(-1,2,2),'F')

    linesMB = np.zeros(int(linesPoints.size/2)).reshape(-1,2)    
    for i in range(0,linesPoints.shape[0]):
        x,y = linesPoints[i]
        mb = np.polyfit(x,y,1)
        linesMB[i] = mb
        
    return(linesMB)

def linesMBtoPolar(linesMB):
    '''convert y=mx+b line to polar form'''
    linesPolar = np.zeros(linesMB.size).reshape(-1,2)
    for i in range(0,linesMB.shape[0]):
        m,b = linesMB[i]
        polar = mb2Hessian(m,b)
        linesPolar[i] = polar
    return(linesPolar)

def indexFromValue(m, mRange, mSamples):
    '''convert value m to sample index in -mRange...mRange over mSample number of samples'''
    return (1+0.5*m)/mRange*mSamples

def valueFromIndex(i, mRange, mSamples):
    '''convert sample index to value in -mRange...mRange over mSample number of samples'''
    return 2*(i*mRange/mSamples-1)

def smoothLinesMB(linesMB):
    '''sort lines into left and right group based on slope, remove lines that are too far from median'''
    mRange = 2
    mSamples = 200
    mVariance = 0.15
    mAccumulator = np.zeros((mSamples), dtype=np.uint64)
    
    for i in range(linesMB.shape[0]):
        oneLineMB = linesMB[i]
        m,b = oneLineMB
        mIndex = int(indexFromValue(m, mRange, mSamples))
        if mIndex<0 or mIndex>=mSamples:
            continue
        mAccumulator[mIndex] +=1

# for testing only        
#     xp = np.linspace(0, mSamples, 1000)
#     _=plt.plot(
#         mAccumulator
#         #linesMB.reshape(-1,2)
#         #x,y,'.',
#         #xp, p(xp), '-'
#     )
#     #plt.ylim(0,540)
#     plt.show()
    
    iMax1=0
    max1=0
    for i in range(0,int(mSamples/2)):
        if (mAccumulator[i]>max1):
            max1=mAccumulator[i]
            iMax1=i
            
    iMax2=0
    max2=0
    for i in range(int(mSamples/2), mSamples):
        if (mAccumulator[i]>max2):
            max2=mAccumulator[i]
            iMax2=i
            
    max1 = valueFromIndex(iMax1, mRange, mSamples)
    max2 = valueFromIndex(iMax2, mRange, mSamples)
    
    lines1 = []
    lines2 = []
    
    for m,b in linesMB:
        if np.abs(m-max1)<mVariance:
            lines1.append([m,b])
        if np.abs(m-max2)<mVariance:
            lines2.append([m,b])
            
    linesMB1 = np.array(lines1)
    linesMB2 = np.array(lines2)
    
    return linesMB1, linesMB2


def xForY(y, m, b):
    '''calculate x from y value of y=mx+b polynomial'''
    return int((y-b)/m)

sLineMBAcc1 = []
sLineMBAcc2 = []
sLineDiff = []

def resetSLineMBAcc():
    '''reset time smoothing accumulators'''
    global sLineMBAcc1
    global sLineMBAcc2
    global sLineDiff
    sLineMBAcc1 = np.zeros(2)
    sLineMBAcc2 = np.zeros(2)
    sLineDiff = []
    

'''default factor for smoothing lines over time'''
smoothing = 0.85

def addSLineMBAcc(sLineMB1, sLineMB2):
    '''smooth lines over time, ignore data too far from accumulator state'''
    global sLineMBAcc1
    global sLineMBAcc2
    global sLineDiff
    
    mVariance = 0.1

    if sLineMBAcc1[0]==0:
        sLineMBAcc1 = sLineMB1
    if sLineMBAcc2[0]==0:
        sLineMBAcc2 = sLineMB2
    if np.abs(sLineMB1[0])<0.01 or np.abs(sLineMB2[0])<0.01 or np.abs(sLineMB1[0])>2 or np.abs(sLineMB2[0])>2:
        return
    if np.abs(sLineMB1[0]-sLineMBAcc1[0])>mVariance:
        return
    if np.abs(sLineMB2[0]-sLineMBAcc2[0])>mVariance:
        return
    sLineDiff.append([np.abs(sLineMB1[0]-sLineMBAcc1[0]), np.abs(sLineMB2[0]-sLineMBAcc2[0])])
    global smoothing
    sLineMBAcc1 = smoothing*sLineMBAcc1 + (1.0-smoothing)*sLineMB1
    sLineMBAcc2 = smoothing*sLineMBAcc2 + (1.0-smoothing)*sLineMB2

def detectLaneLines(img):
    '''detect lane lines pipeline, return calculated images in dict'''
    imgGray = grayscale(img)
    imgBlurred = gaussian_blur(imgGray, 5)
    imgEdges = canny(imgBlurred, 40, 150)

    global x1, x2, x3, x4
    global y14, y23

    vertices = np.array([[(x1, y14), (x2, y23), (x3, y23), (x4, y14)]], dtype=np.int32)
    imgMasked = region_of_interest(imgEdges, vertices)
    
    rho = 4                       # distance resolution in pixels of the Hough grid
    theta_deg = 180
    theta = np.pi/theta_deg       # angular resolution in radians of the Hough grid
    threshold = 32                # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 15          # minimum number of pixels making up a line
    max_line_gap = 7             # maximum gap in pixels between connectable line segments
    
    lines = hough_lines(imgMasked, rho, theta, threshold, min_line_len, max_line_gap)
        
    linesMB = linesPoints2MB(lines)

    linesMB1, linesMB2 = smoothLinesMB(linesMB)
    
    if not linesMB1.size:
        linesMB1=np.zeros([1,2])
    if not linesMB2.size:
        linesMB2=np.zeros([1,2])
        
    sLineMB1=(np.average(linesMB1, axis=0))
    sLineMB2=(np.average(linesMB2, axis=0))
    
    addSLineMBAcc(sLineMB1, sLineMB2)
    
    global sLineMBAcc1
    global sLineMBAcc2
    
    smoothLines = np.array([[[xForY(y14,sLineMBAcc1[0],sLineMBAcc1[1]),y14,xForY(y23,sLineMBAcc1[0],sLineMBAcc1[1]),y23]],
                                [[xForY(y14,sLineMBAcc2[0],sLineMBAcc2[1]),y14,xForY(y23,sLineMBAcc2[0],sLineMBAcc2[1]),y23]]])
    
    imgSmoothLines = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(imgSmoothLines, smoothLines, 3)    

    imgSmoothLinesComp = weighted_img(img, 0.7, imgSmoothLines, 1, 0)
    
    imgHoughLines = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(imgHoughLines, lines, 2)
    
    imgHoughLinesComp = weighted_img(img, 0.7, imgHoughLines, 1, 0)
    
    return { 'gray' : imgGray,
             'blurred' : imgBlurred,
             'edges' : imgEdges,
             'masked': imgMasked,
             'smoothLines': imgSmoothLines,
             'smoothLinesComp': imgSmoothLinesComp,            
             'houghLines': imgHoughLines,
             'houghLinesComp': imgHoughLinesComp }

# for testing only
# img = mpimg.imread('./test_images/solidYellowCurve.jpg')
# _=detectLaneLines(img)

In [21]:
# mask vertices for test images and videos

x1=80
x2=420
x3=555
x4=960
y14=540
y23=330


In [22]:
import os

inputFolder = 'test_images/'
outputFolder = 'test_images_output/'

# process test images

for fileName in os.listdir(inputFolder):
    fileNameSplit = os.path.splitext(fileName)
    
    inFileName = inputFolder+fileName
    print(inFileName)
    img = mpimg.imread(inFileName)
    
    resetSLineMBAcc()
    outImages = detectLaneLines(img)
    
    for imgType in outImages:
        outFileName = '{0}{2}_{1}.jpg'.format(outputFolder, fileNameSplit[0],imgType)
        print(outFileName)
        cmap=None
        if not outImages[imgType].shape==3:
            cmap='gray'
        mpimg.imsave(outFileName, 
                     outImages[imgType],
                     cmap=cmap)
        
    

test_images/solidWhiteCurve.jpg
test_images_output/smoothLines_solidWhiteCurve.jpg
test_images_output/edges_solidWhiteCurve.jpg
test_images_output/blurred_solidWhiteCurve.jpg
test_images_output/masked_solidWhiteCurve.jpg
test_images_output/gray_solidWhiteCurve.jpg
test_images_output/smoothLinesComp_solidWhiteCurve.jpg
test_images_output/houghLines_solidWhiteCurve.jpg
test_images_output/houghLinesComp_solidWhiteCurve.jpg
test_images/solidWhiteRight.jpg
test_images_output/smoothLines_solidWhiteRight.jpg
test_images_output/edges_solidWhiteRight.jpg
test_images_output/blurred_solidWhiteRight.jpg
test_images_output/masked_solidWhiteRight.jpg
test_images_output/gray_solidWhiteRight.jpg
test_images_output/smoothLinesComp_solidWhiteRight.jpg
test_images_output/houghLines_solidWhiteRight.jpg
test_images_output/houghLinesComp_solidWhiteRight.jpg
test_images/solidYellowCurve.jpg
test_images_output/smoothLines_solidYellowCurve.jpg
test_images_output/edges_solidYellowCurve.jpg
test_images_output/bl

In [23]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(image):
    outImages = detectLaneLines(image)
    return outImages['smoothLinesComp']

resetSLineMBAcc()
USE_BLUE_CHANNEL = False
smoothing=0.85
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████▌| 221/222 [00:03<00:00, 61.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 3.98 s


In [24]:
resetSLineMBAcc()
USE_BLUE_CHANNEL = True
smoothing=0.85
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:10<00:00, 61.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 11.3 s


In [25]:
# mask vertices for challenge video

x1=180
x2=530
x3=770
x4=1100
y14=660
y23=470


In [26]:
resetSLineMBAcc()
USE_BLUE_CHANNEL = True
smoothing=0.7
challenge_output = 'test_videos_output/challenge.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:07<00:00, 32.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 8.31 s
